In [1]:
from datasets import DatasetDict, Dataset
from transformers import (
    RobertaTokenizer,
    RobertaForSequenceClassification,
    T5Tokenizer,
    T5ForConditionalGeneration,
    T5Config,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    GenerationConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    EarlyStoppingCallback,
    pipeline,
)
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
import numpy as np
import time
import gc
import GPUtil
import evaluate
from numba import cuda
import wandb
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from functools import partial
import wandb
import os
import pickle
import optuna
from typing import Dict, Union, Optional, Tuple, List, Any
import pandas as pd

2023-11-08 11:00:19.990105: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-08 11:00:19.990158: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-08 11:00:19.990186: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
wandb.login()
os.environ["WAND_NOTEBOOK_NAME"] = "w266_final_project_models"
os.environ["WANDB_DIR"] = "../models/wandb"
os.environ["WANDB_PROJECT"] = "w266_final_project"

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: garykong. Use `wandb login --relogin` to force relogin


In [3]:
# Random seed for reproducibility
RANDOM_SEED = 42

# Parameters for classification
BATCH_SIZE_EVAL = 32
BATCH_SIZE_TRAIN = 32

# Default parameters for T5 model fine-tuning
PER_DEVICE_TRAIN_BATCH_SIZE = 64
PER_DEVICE_EVAL_BATCH_SIZE = 128
LEARNING_RATE = 3e-4
NUM_TRAIN_EPOCHS = 20
EARLY_STOPPING_PATIENCE = 2
NUM_BEAMS = 4

# Setting the DEVICE to cuda
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set path for profane word list
PROFANE_WORD_PATH = "../data/raw/en.txt"

# Set path for raw dataset dictionary
RAW_DATASET_PATH = "../data/processed/raw_dataset.pkl"

# Set maximum length for input and output
MAX_INPUT_LENGTH = 64
MAX_OUTPUT_LENGTH = 64

In [4]:
# Load tokenizers and models
tokenizer_t5_base = T5Tokenizer.from_pretrained("t5-base")
model_t5_base = T5ForConditionalGeneration.from_pretrained("t5-base").to(DEVICE)
tokenizer_t5_small = T5Tokenizer.from_pretrained("t5-small")
model_t5_small = T5ForConditionalGeneration.from_pretrained("t5-small").to(DEVICE)
tokenizer_toxicity = RobertaTokenizer.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier")
model_toxicity = RobertaForSequenceClassification.from_pretrained("SkolkovoInstitute/roberta_toxicity_classifier").to(DEVICE)
tokenizer_acceptability = AutoTokenizer.from_pretrained("iproskurina/tda-bert-en-cola")
model_acceptability = AutoModelForSequenceClassification.from_pretrained("iproskurina/tda-bert-en-cola").to(DEVICE)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5.py:238: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [5]:
# Load dataset
raw_datasets = DatasetDict.load_from_disk(RAW_DATASET_PATH)

## Debugging Functions

In [6]:
def measure_time(func, *args, **kwargs):
    """
    Calculates the time it takes to run a function.
    """
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Function {func.__name__} took {elapsed_time:.2f} seconds to run.")
    return result

def get_gpu_memory():
    """
    Gets the GPU memory information.
    """
    gpus = GPUtil.getGPUs()
    gpu = gpus[0]
    print(f"Total GPU memory: {gpu.memoryTotal}MB")
    print(f"Free GPU memory: {gpu.memoryFree}MB")
    print(f"Used GPU memory: {gpu.memoryUsed}MB")

def force_clear_GPU_memory():
    """
    Force clears the GPU memory.
    """
    cuda.select_device(0)
    cuda.close()

def cleanup():
    """
    Cleans up the GPU memory.
    """
    gc.collect()
    torch.cuda.empty_cache()

## Evaluation Metrics

In [7]:
# Initialize model variables
model_bleurt = None
model_bertscore = None
model_sacrebleu = None

def calc_sacrebleu(refs, preds):
    """
    Calculates the SacreBLEU score.

    Args:
        refs (list): List of reference sentences
        preds (list): List of predicted sentences
    
    Returns:
        results (float): SacreBLEU score
    """
    global model_sacrebleu

    if model_sacrebleu is None:
        model_sacrebleu = evaluate.load("sacrebleu")

    results = model_sacrebleu.compute(predictions=preds, references=refs)["score"]
    results = results/100

    return results

def calc_bert_score(
    refs, preds, model_type="microsoft/deberta-large-mnli", output_mean=True
    ):
    """
    Calculates BERT score per line. Note: https://docs.google.com/spreadsheets/d/1RKOVpselB98Nnh_EOC4A2BYn8_201tmPODpNWu4w7xI/edit#gid=0 lists the best performing models
    Args:
        refs (list): List of reference sentences.
        y_pred (list): List of predicted sentences.
        model_type (str): Type of BERT model to use.
        output_mean (bool): Whether to output the mean of the scores.

    Returns:
        list of precision, recall, f1 scores.

    """
    global model_bertscore

    if model_bertscore is None:
        model_bertscore = evaluate.load("bertscore")
        
    results = model_bertscore.compute(predictions=preds, references=refs, model_type=model_type)
    precision = np.array(results["precision"])
    recall = np.array(results["recall"])
    f1 = np.array(results["f1"])
    
    if output_mean:
        precision = precision.mean()
        recall = recall.mean()
        f1 = f1.mean()

    return precision, recall, f1

def calc_bleurt(refs, preds, checkpoint="BLEURT-20_D12", output_mean = True):
    """
    Calculates BLEURT score per line.

    Args:
        refs (list): List of reference sentences.
        preds (list): List of predicted sentences.
        output_type (str): Type of output to return. Either 'numpy' or 'list'.

    Returns:
        list/array of BLEURT scores.
    """
    global model_bleurt

    if model_bleurt is None:
        model_bleurt = evaluate.load("bleurt", module_type="metric", checkpoint=checkpoint)

    results = np.array(model_bleurt.compute(predictions=preds, references=refs)["scores"])

    if output_mean:
        results = results.mean()

    return results

def calc_tox_acceptability(
    data,
    tokenizer,
    model,
    output_score=True,
    output_mean=True):
    """
    Calculates toxicity and acceptability scores for a given dataset.

    Args:
        data = list of strings to be evaluated
        tokenizer = tokenizer for the model
        model = model to be used for evaluation
        output_score = whether to output the score or the label
        output_mean = whether to output the mean of the scores or the scores for each sentence
    
    Returns:
        array of toxicity and acceptability scores.
    """  
    inputs = tokenizer(data, return_tensors="pt", padding=True).to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs)["logits"]
        if output_score:
            result = torch.nn.functional.softmax(logits, dim=1)[:, 1]
        else:
            result = logits.argmax(1).data
        result = result.cpu().numpy()

    if output_mean:
        result = result.mean()
        
    return result

def evaluate_metrics(
    refs,
    preds,
    tokenizer_toxicity=tokenizer_toxicity,
    model_toxicity=model_toxicity,
    tokenizer_acceptability=tokenizer_acceptability,
    model_acceptability=model_acceptability,
    to_neutral=True,
    weights={
        "BLEU": 0.2,
        "STA": 0.4,
        "Acceptability": 0.2,
        "BERT_Score": 0.2
    },
    include_bleurt=False
):
    """
    Calculates and returns a dictionary of evaluation metrics

    Args:
        refs (list): list of strings (reference)
        preds (list): list of strings (predictions)
        tokenizer_toxicity (tokenizer): tokenizer for toxicity model
        model_toxicity (model): toxicity model
        tokenizer_acceptability (tokenizer): tokenizer for acceptability model
        model_acceptability (model): acceptability model
        to_neutral (bool): whether the goal is to transfer to neutral (True) or to toxic (False)
        weights (dict): dictionary of weights for each metric
        include_bleurt (bool): whether to include BLEURT score in the output

    Returns:
        results (dict): dictionary of evaluation metrics
    """

    # Calculate BLEU score
    bleu = calc_sacrebleu(refs, preds)

    # Calculate toxicity classification
    # tox_ref = calc_tox_acceptability(refs, tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)
    tox_pred = calc_tox_acceptability(preds, tokenizer_toxicity, model_toxicity, output_score=False, output_mean=False)

    # Calculate style transfer accuracy as proportion of sentences that were correctly classified (as non-toxic / toxic)
    if to_neutral:
        sta_correct_label = 0
    else:
        sta_correct_label = 1

    # sta_ref = (tox_ref == sta_correct_label).sum() / len(tox_ref)
    sta_pred = (tox_pred == sta_correct_label).sum() / len(tox_pred)
    # sta_pct = sta_pred / sta_ref

    # Calculate acceptability scores
    # acc_ref = calc_tox_acceptability(refs, tokenizer_acceptability, model_acceptability)
    acc_pred = calc_tox_acceptability(preds, tokenizer_acceptability, model_acceptability)
    # acc_pct = acc_pred / acc_ref

    # Calculate similarity score
    bert_score_f1 = calc_bert_score(refs, preds, model_type="distilbert-base-uncased")[2]

    # Calculate BLEURT score if include_bleurt is True
    bleurt = None
    if include_bleurt:
        bleurt = calc_bleurt(refs, preds)

    # Calculate composite score
    composite_score = weights["BLEU"] * bleu + weights["STA"] * sta_pred + weights["Acceptability"] * acc_pred + weights["BERT_Score"] * bert_score_f1

    # Return a dictionary of metrics
    results = {
        "BLEU": bleu,
        "STA": sta_pred,
        # "STA_pct": sta_pct,
        "FLU": acc_pred,
        # "Acceptability_pct": acc_pct,
        "SEM": bert_score_f1,
        "Overall": composite_score,
    }
    if include_bleurt:
        results["BLEURT"] = bleurt
        
    return results


# Baseline Models

In [8]:
def baseline_detoxifier(text_list, profane_word_path=PROFANE_WORD_PATH):
    """
    Returns a detoxified version of the text by replacing toxic terms with blanks

    Args:
        text_list (list): list of strings to be detoxified
        toxic_list (list): list of toxic terms to be removed from text_list

    Returns:
        detoxified_text_list (list): list of detoxified strings
    """
    # Load list of profane words
    profane_words = []
    with open(profane_word_path, "r") as f:
        for line in f:
            profane_words.append(line.strip())

    # Detoxify text
    y_pred_delete = []
    for text in text_list:
        for term in profane_words:
            text = text.replace(term, "")
        y_pred_delete.append(text)

    return y_pred_delete

def bart_detoxifier(text_list):
    """
    Returns a detoxified version of the text using BART

    Args:
        text_list (list): list of strings to be detoxified

    Returns:
        detoxified_text_list (list): list of detoxified strings
    """
    # Create predictions using BART
    pipe_bart = pipeline("text2text-generation", model="s-nlp/bart-base-detox", device=DEVICE)

    # Create predictions using BART and show progress using tqdm
    y_pred_bart = pipe_bart(text_list, max_length=MAX_OUTPUT_LENGTH, truncation=True)

    # Convert to list of strings
    y_pred_bart = [x["generated_text"] for x in y_pred_bart]
    
    return y_pred_bart

In [9]:
# Evaluate DELETE model on validation set
delete_preds_val = baseline_detoxifier(raw_datasets["validation"]['source'])
delete_val_metrics = evaluate_metrics(raw_datasets["validation"]['target'], delete_preds_val)
delete_val_metrics

NameError: name 'y_pred_delete' is not defined

In [ ]:
# Evaluate BART model on validation set
bart_preds_val = bart_detoxifier(raw_datasets["validation"]['source'])
bart_val_metrics = evaluate_metrics(raw_datasets["validation"]['target'], bart_preds_val)
bart_val_metrics

{'BLEU': 0.7015951162845684,
 'STA_preds': 0.9178541492036881,
 'Acceptability_preds': 0.71802455,
 'BERT_score_f1': 0.9451393306205379,
 'Overall': 0.8400934594361843}

# Helper Functions to Fine-tune T5 Models

In [8]:
def add_prefix(datasetdict, prefix="to_neutral: "):
    """Adds a prefix to the source sequence in the dataset."""
    datasetdict_copy = datasetdict.copy()
    datasetdict_copy["train"] = datasetdict_copy["train"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy["validation"] = datasetdict_copy["validation"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy["test"] = datasetdict_copy["test"].map(lambda x: {"source": prefix + x["source"]})
    datasetdict_copy = DatasetDict(datasetdict_copy)
    return datasetdict_copy

def preprocess_function(examples, tokenizer):
    """Preprocess function for T5."""
    model_inputs = tokenizer(
        examples["source"],
        text_target=examples["target"],
        max_length=MAX_INPUT_LENGTH,
        truncation=True,
    )
    return model_inputs

def post_process(preds, refs, tokenizer):
    """
    Post-process function for T5.

    Args:
        preds (list): list of predicted sequences
        refs (list): list of reference sequences
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding

    Returns:
        decoded_preds (list): list of decoded predicted sequences
        decoded_refs (list): list of decoded reference sequences
    """
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    refs = np.where(refs != -100, refs, tokenizer.pad_token_id)
    decoded_refs = tokenizer.batch_decode(refs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_refs = [ref.strip() for ref in decoded_refs]

    return decoded_preds, decoded_refs

def compute_metrics(eval_preds, tokenizer):
    """
    Function to calculate the metrics for trainer.evaluate().

    Args:
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding the predictions
        eval_preds (tuple): Tuple containing the predictions and references

    Returns:
        dict: Dictionary containing the metrics
    """
    preds, refs = eval_preds

    # Post-process the predictions and references
    decoded_preds, decoded_refs = post_process(preds, refs, tokenizer)
    
    # Evaluate metrics
    return evaluate_metrics(
        decoded_refs,
        decoded_preds,
        tokenizer_toxicity=tokenizer_toxicity,
        model_toxicity=model_toxicity,
        tokenizer_acceptability=tokenizer_acceptability,
        model_acceptability=model_acceptability,
        include_bleurt=False
    )

def setup_trainer(output_dir_name,
                train_dataset,
                eval_dataset,
                model_checkpoint="t5-small",
                per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                num_train_epochs=NUM_TRAIN_EPOCHS,
                max_length=MAX_OUTPUT_LENGTH,
                num_beams=NUM_BEAMS,
                early_stopping_patience=EARLY_STOPPING_PATIENCE,
                report_to="wandb",
                ):
    """
    Set up a Seq2SeqTrainer object for training a T5 model.

    Default parameters based on this: https://github.com/google-research/text-to-text-transfer-transformer/blob/main/t5/models/hf_model.py#L55

    Args:
        output_dir_name (str): What to name the model in the output directory.
        model_checkpoint (str): Name of the pre-trained model to use.
        train_dataset (Dataset): Training dataset.
        eval_dataset (Dataset): Validation/test dataset.
        per_device_train_batch_size (int): Batch size for training.
        per_device_eval_batch_size (int): Batch size for evaluation.
        learning_rate (float): Learning rate for optimizer.
        weight_decay (float): Weight decay for optimizer.
        num_train_epochs (int): Number of training epochs.
        max_length (int): Maximum length of generated sequences.
        num_beams (int): Number of beams for beam search.
        compute_metrics (function): Function to compute evaluation metrics.
        callbacks (list): List of callbacks to use.
        trainer_class (Seq2SeqTrainer): Trainer class to use.

    Returns:
        Seq2SeqTrainer: Trainer object for training the T5 model.
    """
    
    # Instantiate model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

    # Define the data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model, return_tensors="pt", padding=True)

    # Define generation config
    generation_config = GenerationConfig(
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        eos_token_id=model.config.eos_token_id,
        bos_token_id=model.config.bos_token_id,
        pad_token_id=model.config.pad_token_id,
        decoder_start_token_id=model.config.pad_token_id
        )

    # Save the generation config
    gen_config_path = f"../models/{output_dir_name}/generation_config"
    generation_config.save_pretrained(gen_config_path)

    # Define the training arguments
    args = Seq2SeqTrainingArguments(
        output_dir=f'../models/{output_dir_name}',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        learning_rate=learning_rate, 
        predict_with_generate=True,
        generation_config=gen_config_path,
        fp16=True,
        report_to=report_to,
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="Overall",
        greater_is_better=True,
        generation_max_length=max_length,
    )

    # Create a partial function with the tokenizer argument included
    compute_metrics_with_tokenizer = partial(compute_metrics, tokenizer=tokenizer)
    
    # Instantiate the trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics_with_tokenizer,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]

    )

    return trainer

def create_bidirectional_dataset(datasets, shuffle=True):
    """
    Creates a bi-directional dataset from the original dataset.

    Args:
        datasets (DatasetDict): DatasetDict object containing the original dataset.
        shuffle (bool): Whether to shuffle the dataset or not.
    
    Returns:
        extended_datasets (DatasetDict): DatasetDict object containing the bi-directional dataset.
    """

    def bidirectional_extension(dataset):
        new_data = {
            "source": [],
            "target": []
        }
        for src, tgt in zip(dataset['source'], dataset['target']):
            new_data['source'].extend([f'to_neutral: {src}', f'to_toxic: {tgt}'])
            new_data['target'].extend([tgt, src])
        return new_data

    extended_train_data = bidirectional_extension(datasets["train"])
    extended_validation_data = bidirectional_extension(datasets["validation"])
    extended_test_data = bidirectional_extension(datasets["test"])

    extended_datasets = DatasetDict({
        "train": Dataset.from_dict(extended_train_data),
        "validation": Dataset.from_dict(extended_validation_data),
        "test": Dataset.from_dict(extended_test_data)
    })

    if shuffle:
        extended_datasets["train"] = extended_datasets["train"].shuffle(seed=RANDOM_SEED)
        
    return extended_datasets

def compute_metrics_bd(eval_preds, tokenizer, bd_dataset, shuffled_data=False):
    """
    Function to calculate the metrics for trainer.evaluate().
    This function is for the bi-directional model.
    
    Args:
        eval_preds (tuple): Tuple containing the predictions and references
        tokenizer (PreTrainedTokenizer): tokenizer to use for decoding the predictions
        shuffled_data (bool): Whether the data is shuffled or not
        bd_dataset (DatasetDict): Bidirectional dataset to use for testing created using create_bidirectional_datasets
                                  For example, raw_datasets_bd["validation"] or raw_datasets_bd["test"]

    Returns:
        dict: Dictionary containing the metrics
    """
    preds, refs = eval_preds

    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    refs = np.where(refs != -100, refs, tokenizer.pad_token_id)
    decoded_refs = tokenizer.batch_decode(refs, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_refs = [ref.strip() for ref in decoded_refs]
    
    # If shuffled data is false, have to_neutral_preds and to_neutral_refs just be predictions and refs with even indices
    if not shuffled_data:
        to_neutral_preds = decoded_preds[::2]
        to_neutral_refs = decoded_refs[::2]
    # Otherwise, get the indices to use when splitting predictions and refs to to_neutral and to_toxic
    else:
        # Get the indices to use when splitting predictions and refs to to_neutral and to_toxic
        to_neutral_idx = [i for i, input_sentence in enumerate(bd_dataset['source']) if input_sentence.startswith("to_neutral")]

        # Retrieve based on the indices
        to_neutral_preds = [decoded_preds[i] for i in to_neutral_idx]
        to_neutral_refs = [decoded_refs[i] for i in to_neutral_idx]
    
    # Evaluate metrics for to_neutral
    to_neutral_metrics = evaluate_metrics(
        to_neutral_refs,
        to_neutral_preds,
        to_neutral=True
    )

    # Return dictionary of to_neutral metrics
    return to_neutral_metrics

# Fine-tune T5 (Unidirectional)

In [10]:
prefixed_datasets = add_prefix(raw_datasets)

tokenized_datasets_t5_small = prefixed_datasets.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small = setup_trainer(
    output_dir_name="t5-small-detoxify-2",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_t5_small["train"],
    eval_dataset=tokenized_datasets_t5_small["validation"],
)

wandb.init(project="w266_final_project", name="t5-small-detoxify-2")
trainer_t5_small.train()
wandb.finish()

NameError: name 'add_prefix' is not defined

# Fine-tune T5 Model (Bi-directional, No custom loss)

### Trial without shuffled data

In [103]:
raw_datasets_bd = create_bidirectional_dataset(raw_datasets, shuffle=False)

tokenized_datasets_bd_t5_small = raw_datasets_bd.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small_bd = setup_trainer(
    output_dir_name="t5-small-detoxify-bd-noshuffle-2",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_bd_t5_small["train"],
    eval_dataset=tokenized_datasets_bd_t5_small["validation"],
    compute_metrics=partial(compute_metrics_bd, bd_dataset=raw_datasets_bd["validation"], shuffled_data=False)
    )

wandb.init(project="w266_final_project", name="t5-small-bd-noshuffle-detoxify-2")
trainer_t5_small_bd.train()
wandb.finish()

Map:   0%|          | 0/21466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2386 [00:00<?, ? examples/s]

Map:   0%|          | 0/1342 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Bleu,Sta Preds,Acceptability Preds,Bert Score F1,Overall
1,1.487700,1.235429,0.589117,0.784577,0.678174,0.923197,0.751928
2,1.257100,1.167136,0.599725,0.836547,0.691290,0.925403,0.777902
3,1.171800,1.151006,0.600607,0.860855,0.689214,0.925960,0.787498
4,1.110400,1.143344,0.605002,0.892707,0.692520,0.926318,0.801851
5,1.063400,1.128787,0.604396,0.903604,0.704711,0.926202,0.808504
6,1.024400,1.140509,0.602471,0.898575,0.700976,0.925787,0.805277
7,0.989600,1.132926,0.601382,0.909472,0.705822,0.925794,0.810388
8,0.956400,1.134517,0.600770,0.900251,0.702148,0.925799,0.805844
9,0.930500,1.137347,0.604801,0.901090,0.699633,0.926039,0.806531


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/Acceptability_preds,▁▄▄▅█▇█▇▆
eval/BERT_score_f1,▁▆▇██▇▇▇▇
eval/BLEU,▁▆▆██▇▆▆█
eval/Overall,▁▄▅▇█▇█▇█
eval/STA_preds,▁▄▅▇█▇█▇█
eval/loss,█▄▂▂▁▂▁▁▂
eval/runtime,▇█▄▇▆▁▆▃▃
eval/samples_per_second,▂▁▄▂▃█▃▆▆
eval/steps_per_second,▂▁▄▂▃█▃▆▆
train/epoch,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇███
train/global_step,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▇▇███


### Trial with shuffled data

In [102]:
raw_datasets_bd_shuffle = create_bidirectional_dataset(raw_datasets, shuffle=True)

tokenized_datasets_bd_shuffle_t5_small = raw_datasets_bd_shuffle.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small_bd_shuffle = setup_trainer(
    output_dir_name="t5-small-detoxify-bd-shuffle-2",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_bd_shuffle_t5_small["train"],
    eval_dataset=tokenized_datasets_bd_shuffle_t5_small["validation"],
    compute_metrics=partial(compute_metrics_bd, bd_dataset=raw_datasets_bd_shuffle["validation"], shuffled_data=True)
    )

wandb.init(project="w266_final_project", name="t5-small-bd-shuffle-detoxify-2")
trainer_t5_small_bd_shuffle.train()
wandb.finish()

Map:   0%|          | 0/21466 [00:00<?, ? examples/s]

Map:   0%|          | 0/2386 [00:00<?, ? examples/s]

Map:   0%|          | 0/1342 [00:00<?, ? examples/s]

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


Epoch,Training Loss,Validation Loss,Bleu,Sta Preds,Acceptability Preds,Bert Score F1,Overall
1,1.495300,1.230033,0.596087,0.780386,0.690938,0.922194,0.753998
2,1.257900,1.177033,0.598875,0.854987,0.690168,0.923962,0.784596
3,1.171200,1.142365,0.599957,0.891869,0.720199,0.923604,0.805500
4,1.110500,1.137581,0.606112,0.881811,0.699919,0.925512,0.799033
5,1.061300,1.134472,0.603866,0.896060,0.701756,0.925853,0.804719


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/Acceptability_preds,▁▁█▃▄
eval/BERT_score_f1,▁▄▄▇█
eval/BLEU,▁▃▄█▆
eval/Overall,▁▅█▇█
eval/STA_preds,▁▆█▇█
eval/loss,█▄▂▁▁
eval/runtime,█▂▆▅▁
eval/samples_per_second,▁▇▃▄█
eval/steps_per_second,▁▆▃▄█
train/epoch,▁▁▃▃▅▅▆▆███
train/global_step,▁▁▃▃▅▅▆▆███


# Fine-tune T5 Model (Custom Loss)

Documentation for original loss function:

https://huggingface.co/transformers/v4.2.2/_modules/transformers/trainer.html#Trainer.compute_loss

In [21]:
class Seq2SeqTrainingArgumentsCustomLoss(Seq2SeqTrainingArguments):
    def __init__(self, classifier_loss_weight, bidirectional, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.classifier_loss_weight = classifier_loss_weight
        self.bidirectional = bidirectional

class Seq2SeqTrainerCustomLoss(Seq2SeqTrainer):
    def __init__(self, model_toxicity, tokenizer_toxicity, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.classifier_loss_weight = self.args.classifier_loss_weight
        self.model_toxicity = model_toxicity.to(self.model.device)
        self.tokenizer_toxicity = tokenizer_toxicity
        self.loss_fct = torch.nn.CrossEntropyLoss().to(self.model.device)
        self.bidirectional = self.args.bidirectional

    def generate_pred_tokens(
            self,
            inputs: Dict[str, Union[torch.Tensor, Any]],
            **gen_kwargs,
        ):        
        # Prepare the inputs
        inputs = self._prepare_inputs(inputs)
        
        # Merge default generation kwargs with any user-provided kwargs
        if len(gen_kwargs) == 0 and hasattr(self, "_gen_kwargs"):
            gen_kwargs = self._gen_kwargs.copy()
        
        # Set 'synced_gpus' to False in gen_kwargs
        gen_kwargs["synced_gpus"] = False
        
        # Copy inputs to a new dictionary for generation
        generation_inputs = inputs.copy()
        
        # Remove 'decoder_input_ids' if it was created from 'labels'
        if ("labels" in generation_inputs and 
            "decoder_input_ids" in generation_inputs and 
            generation_inputs["labels"].shape == generation_inputs["decoder_input_ids"].shape):
            generation_inputs = {k: v for k, v in inputs.items() if k != "decoder_input_ids"}
        
        # Generate tokens using the model
        generated_tokens = self.model.generate(**generation_inputs, **gen_kwargs)
               
        return generated_tokens

    def classifier_guided_loss(self, generated_text, target_labels):
        # Tokenize the generated text
        preds_tokenized = self.tokenizer_toxicity(generated_text, return_tensors="pt", padding=True)
        # print(f"Prediction tokens: {preds_tokenized}")
        
        # Move inputs to the same device as the toxicity model
        preds_tokenized = {key: tensor.to(self.model_toxicity.device) for key, tensor in preds_tokenized.items()}

        # Get the classifier's output logits
        logits = self.model_toxicity(**preds_tokenized)["logits"]
        # print(f"Logits: {logits}")

        # Calculate the cross-entropy loss
        loss = self.loss_fct(logits, target_labels)
        # print(f"Classifier-guided loss: {loss}")
        
        return loss

    def compute_loss(self, model, inputs):
        # Get outputs, predictions and prediction loss
        outputs = model(**inputs)
        pred_loss = outputs.loss if isinstance(outputs, dict) else outputs[0]
        # print(f"Prediction loss: {pred_loss}")

        # Call prediction_step
        preds = self.generate_pred_tokens(inputs)
        # print(f"Prediction tokens: {preds}")

        # Decode predictions
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = self.tokenizer.batch_decode(preds, skip_special_tokens=True)
        decoded_preds = [pred.strip() for pred in decoded_preds]
        # print(f"Decoded predictions: {decoded_preds}")

        # If bidirectional:
        if self.bidirectional:
            # Identify indices corresponding to <to_neutral> and <to_toxic>
            decoded_inputs = self.tokenizer.batch_decode(inputs["input_ids"], skip_special_tokens=True)

            # Convert decoded_inputs to a NumPy array
            decoded_inputs_np = np.array(decoded_inputs)

            # Vectorized operation to identify indices
            to_neutral_mask = np.char.startswith(decoded_inputs_np, "to_neutral")
            to_toxic_mask = np.char.startswith(decoded_inputs_np, "to_toxic")

            # Get the indices where the conditions are True
            to_neutral_idx = np.where(to_neutral_mask)[0].tolist()
            to_toxic_idx = np.where(to_toxic_mask)[0].tolist()

            # Calculate target labels based on indices. The length should be the same as the number of predictions
            target_labels = torch.zeros(len(decoded_preds), dtype=torch.long, device=self.model_toxicity.device)
            target_labels[to_neutral_idx] = 0
            target_labels[to_toxic_idx] = 1

        # If not bidirectional:
        else:
            target_labels = torch.zeros(len(decoded_preds), dtype=torch.long, device=self.model_toxicity.device)
            # print(f"Target labels: {target_labels}")

        # Calculate the classifier-guided loss
        classifier_loss = self.classifier_guided_loss(decoded_preds, target_labels)
        # print(f"Classifier loss: {classifier_loss}")

        # Calculate total loss as a weighted sum of the classifier loss and the prediction loss
        total_loss = self.classifier_loss_weight * classifier_loss + (1 - self.classifier_loss_weight) * pred_loss

        # # Print losses
        # print(f"Classifier loss: {classifier_loss}, Prediction loss: {pred_loss}, Total loss: {total_loss}")
        
        return total_loss
    
def setup_trainer_customloss(output_dir_name,
                train_dataset,
                eval_dataset,
                model_checkpoint="t5-small",
                per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                learning_rate=LEARNING_RATE,
                num_train_epochs=NUM_TRAIN_EPOCHS,
                max_length=MAX_OUTPUT_LENGTH,
                num_beams=NUM_BEAMS,
                early_stopping_patience=EARLY_STOPPING_PATIENCE,
                classifier_loss_weight=1,
                bidirectional=False,
                ):
    
    # Instantiate model and tokenizer
    model = T5ForConditionalGeneration.from_pretrained(model_checkpoint).to(DEVICE)
    tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

    # Define the data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, model, return_tensors="pt", padding=True)

    # Define generation config
    generation_config = GenerationConfig(
        max_length=max_length,
        num_beams=num_beams,
        early_stopping=True,
        eos_token_id=model.config.eos_token_id,
        bos_token_id=model.config.bos_token_id,
        pad_token_id=model.config.pad_token_id,
        decoder_start_token_id=model.config.pad_token_id
        )

    # Save the generation config
    gen_config_path = f"../models/{output_dir_name}/generation_config"
    generation_config.save_pretrained(gen_config_path)

    # Define the training arguments
    args = Seq2SeqTrainingArgumentsCustomLoss(
        output_dir=f'../models/{output_dir_name}',
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_eval_batch_size,
        learning_rate=learning_rate, 
        predict_with_generate=True,
        generation_config=gen_config_path,
        fp16=True,
        report_to="wandb",
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="Overall",
        greater_is_better=True,
        generation_max_length=max_length,
        classifier_loss_weight=classifier_loss_weight,
        bidirectional=bidirectional,
    )
    
    # Instantiate the trainer
    trainer = Seq2SeqTrainerCustomLoss(
        model=model,
        tokenizer=tokenizer,
        model_toxicity=model_toxicity,
        tokenizer_toxicity=tokenizer_toxicity,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
        compute_metrics=partial(compute_metrics, tokenizer=tokenizer),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=early_stopping_patience)]
    )

    return trainer

In [26]:
# Create subset of data for testing custom loss function
raw_datasets_subset = DatasetDict({
    "train": raw_datasets["train"].select(range(1000)),
    "validation": raw_datasets["validation"].select(range(100)),
    "test": raw_datasets["test"]
})

# Add prefix to subset of data
prefixed_datasets_subset = add_prefix(raw_datasets_subset)

# Tokenize subset of data
tokenized_datasets_subset_t5_small = prefixed_datasets_subset.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small_cl_2 = setup_trainer_customloss(
    output_dir_name="t5-small-detoxify-cl",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_subset_t5_small["train"],
    eval_dataset=tokenized_datasets_subset_t5_small["validation"],
    bidirectional=False,
    classifier_loss_weight=1,
    learning_rate=5e-5,
)

# wandb.init(project="w266_final_project", name="t5-small-detoxify-cl-1.0")
trainer_t5_small_cl_2.train()
# wandb.finish()

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,4.306800,2.120496,0.307144,0.080000,0.501987,0.849350,0.363696
2,4.357300,2.120496,0.307144,0.080000,0.501987,0.849350,0.363696
3,4.346700,2.120496,0.307144,0.080000,0.501987,0.849350,0.363696


TrainOutput(global_step=48, training_loss=4.336916605631511, metrics={'train_runtime': 157.696, 'train_samples_per_second': 126.826, 'train_steps_per_second': 2.029, 'total_flos': 30771227394048.0, 'train_loss': 4.336916605631511, 'epoch': 3.0})

In [11]:
trainer_t5_small_cl_1 = setup_trainer_customloss(
    output_dir_name="t5-small-detoxify-cl",
    model_checkpoint="t5-small",
    train_dataset=tokenized_datasets_t5_small["train"],
    eval_dataset=tokenized_datasets_t5_small["validation"],
    bidirectional=False,
    classifier_loss_weight=0.5,
)

wandb.init(project="w266_final_project", name="t5-small-detoxify-cl-0.5")
trainer_t5_small_cl_1.train()
wandb.finish()

Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,1.460600,0.953918,0.589352,0.841576,0.694928,0.921716,0.777830
2,0.915100,0.919530,0.599858,0.875105,0.711380,0.924862,0.797262
3,0.794900,0.918505,0.600539,0.864208,0.701296,0.925812,0.791212
4,0.727600,0.916791,0.604893,0.897737,0.710006,0.926130,0.807300
5,0.679000,0.923304,0.603201,0.888516,0.708156,0.926516,0.802981
6,0.649100,0.918918,0.604479,0.888516,0.709488,0.926689,0.803538


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


eval/BLEU,▁▆▆█▇█
eval/FLU,▁█▄▇▇▇
eval/Overall,▁▆▄█▇▇
eval/SEM,▁▅▇▇██
eval/STA,▁▅▄█▇▇
eval/loss,█▂▁▁▂▁
eval/runtime,▇▁▄█▄▅
eval/samples_per_second,▂█▅▁▅▄
eval/steps_per_second,▂█▅▁▅▅
train/epoch,▁▁▂▂▄▄▅▅▇▇███
train/global_step,▁▁▂▂▄▄▅▅▇▇███


Epoch,Training Loss,Validation Loss,Bleu,Sta,Flu,Sem,Overall
1,4.280300,2.096896,0.323958,0.084661,0.485807,0.853787,0.366575
2,4.298700,2.096896,0.323958,0.084661,0.485807,0.853787,0.366575


KeyboardInterrupt: 

# Evaluation

In [22]:
# Create a pandas dataframe to store evaluation metrics for each model configuration
eval_metrics_df = pd.DataFrame(columns=["Model", "BLEURT", "BLEU", "STA", "FLU", "SEM", "Overall"])
eval_metrics_df

,Model,BLEURT,BLEU,STA,FLU,SEM,Overall


In [24]:
# Helper function to add metrics to the dataframe
def add_metrics_to_df(df, model_name, metrics):
    """
    Add model metrics to a pandas dataframe
    
    Args:
    - df: pandas dataframe to add metrics to
    - model_name: name of the model
    - metrics: dictionary of evaluation metrics
    
    Returns:
    - updated pandas dataframe
    """

    # Check if the model name already exists in the dataframe
    if model_name in df["Model"].values:
        print(f"Model {model_name} already exists in the dataframe.")
        return df
    
    # Add the new row to the dataframe
    model_metrics_df = pd.DataFrame({
        "Model": [model_name],
        "BLEURT": [metrics["BLEURT"]],
        "BLEU": [metrics["BLEU"]],
        "STA": [metrics["STA"]],
        "FLU": [metrics["FLU"]],
        "SEM": [metrics["SEM"]],
        "Overall": [metrics["Overall"]]
    })
    
    return pd.concat([df, model_metrics_df], ignore_index=True)

DELETE

In [25]:
delete_eval_preds = baseline_detoxifier(raw_datasets["test"]["source"])
delete_eval_metrics = evaluate_metrics(raw_datasets["test"]["target"],
                               delete_eval_preds,
                               include_bleurt=True)
print(delete_eval_metrics)
eval_metrics_df = add_metrics_to_df(eval_metrics_df, "DELETE", delete_eval_metrics)

{'BLEU': 0.5036836653565926, 'STA': 0.6184798807749627, 'FLU': 0.5259524, 'SEM': 0.9074216935389443, 'Overall': 0.6348035038444941, 'BLEURT': -0.23698500004748296}


/tmp/ipykernel_13154/2328492495.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, model_metrics_df], ignore_index=True)


BART

In [26]:
bart_eval_preds = bart_detoxifier(raw_datasets["test"]["source"])
bart_eval_metrics = evaluate_metrics(raw_datasets["test"]["target"],
                                     bart_eval_preds,
                                     include_bleurt=True)
print(bart_eval_metrics)
eval_metrics_df = add_metrics_to_df(eval_metrics_df, "BART", bart_eval_metrics)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'BLEU': 0.5618694307428573, 'STA': 0.8926974664679582, 'FLU': 0.7746871, 'SEM': 0.9232455269945716, 'Overall': 0.8090394004102123, 'BLEURT': 0.2596495511168517}


T5 Small (Unidirectional)

In [30]:
prefixed_datasets = add_prefix(raw_datasets)

tokenized_datasets_t5_small = prefixed_datasets.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small_best = setup_trainer(
    output_dir_name="t5-small-detoxify-2",
    model_checkpoint="../models/t5-small-detoxify-2/checkpoint-840",
    train_dataset=tokenized_datasets_t5_small["train"],
    eval_dataset=tokenized_datasets_t5_small["test"],
    report_to=None,
)

t5_small_eval_preds = trainer_t5_small_best.predict(tokenized_datasets_t5_small["test"]).predictions
t5_small_eval_preds = tokenizer_t5_small.batch_decode(t5_small_eval_preds, skip_special_tokens=True)
t5_small_eval_preds = [pred.strip() for pred in t5_small_eval_preds]

t5_small_eval_metrics = evaluate_metrics(raw_datasets["test"]["target"],
                                            t5_small_eval_preds,
                                            include_bleurt=True)

print(t5_small_eval_metrics)

eval_metrics_df = add_metrics_to_df(eval_metrics_df, "T5-UD", t5_small_eval_metrics)

Map:   0%|          | 0/10733 [00:00<?, ? examples/s]

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/671 [00:00<?, ? examples/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB (GPU 0; 21.99 GiB total capacity; 4.57 GiB already allocated; 27.69 MiB free; 5.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

T5 Small (Bidirectional)

In [ ]:
raw_datasets_bd = create_bidirectional_dataset(raw_datasets, shuffle=False)

tokenized_datasets_bd_t5_small = raw_datasets_bd.map(
    preprocess_function,
    fn_kwargs={'tokenizer': tokenizer_t5_small},
    batched=True,
    remove_columns=["source", "target"],
)

trainer_t5_small_bd_best = setup_trainer(
    output_dir_name="t5-small-detoxify-bd-noshuffle-2",
    model_checkpoint="../models/t5-small-detoxify-bd-noshuffle-2/checkpoint-2352",
    train_dataset=tokenized_datasets_bd_t5_small["train"],
    eval_dataset=tokenized_datasets_bd_t5_small["test"],
    compute_metrics=partial(compute_metrics_bd, bd_dataset=raw_datasets_bd["test"], shuffled_data=False)
    report_to=None,
    )

t5_small_bd_eval_preds = trainer_t5_small_bd_best.predict(tokenized_datasets_t5_small["test"]).predictions
t5_small_bd_eval_preds = tokenizer_t5_small.batch_decode(t5_small_bd_eval_preds, skip_special_tokens=True)
t5_small_bd_eval_preds = [pred.strip() for pred in t5_small_bd_eval_preds]

t5_small_bd_eval_metrics = evaluate_metrics(raw_datasets["test"]["target"],
                                            t5_small_bd_eval_preds,
                                            include_bleurt=True)

print(t5_small_bd_eval_metrics)

eval_metrics_df = add_metrics_to_df(eval_metrics_df, "T5-BD", t5_small_bd_eval_metrics)